# Digit Classification with Neural Networks 

Interest in neural networks, and in particular those with architechures that support deep learning, has been surging in recent years.  

In this notebook we will be revisiting the problem of digit classification on the MNIST data.  In doing so, we will introduce a new Python library, Theano, for working with neural networks.  Theano is a popular choice for neural networks as the same code can be run on either CPUs or GPUs.  GPUs greatly speed up the training and prediction, and is readily available. Amazon even offers GPU machines on EC2.  

In part 1, we'll introduce Theano, and refresh ourselves on the MNIST dataset.  In part 2, we'll create a multi-layer neural network with a simple architechure, and train it using backpropagation.  Part 3 will introduce the convolutional architechure, which can be said to be doing 'deep learning' (also called feature learning or representation learning).

#### Part 1: Basics

Lets start to look at Theano.  If later you'd like to go deeper into Theano, you may want to read this paper: http://www.iro.umontreal.ca/~lisa/pointeurs/theano_scipy2010.pdf

Install Theano if you haven't already.  Then let's load it, and set it to work with a CPU.  For reference, here is the Theano documentation: http://www.deeplearning.net/software/theano/library/

In [1]:
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import time

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print (theano.config.device) # We're using CPUs (for now)
print (theano.config.floatX) # Should be 64 bit for CPUs

np.random.seed(0)

cpu
float64


Now for MNIST data...

In [2]:
# Repeating steps from Project 1 to prepare mnist dataset. 
mnist = fetch_mldata('MNIST original', data_home='~/datasets/mnist')
X, Y = mnist.data, mnist.target
X = X / 255.0
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]
numExamples = 2000
test_data, test_labels = X[70000-numExamples:], Y[70000-numExamples:]
train_data, train_labels = X[:numExamples], Y[:numExamples]
numFeatures = train_data[1].size
numTrainExamples = train_data.shape[0]
numTestExamples = test_data.shape[0]
print ('Features = %d' %(numFeatures))
print ('Train set = %d' %(numTrainExamples))
print ('Test set = %d' %(numTestExamples))

Features = 784
Train set = 2000
Test set = 2000


Looking ahead to working with neural networks, let's prepare one additional variation of the label data.  Let's make these labels, rather than each being an integer value from 0-9, be a set of 10 binary values, one for each class.  This is sometimes called a 1-of-n encoding, and it makes working with Neural Networks easier, as there will be one output node for each class.

In [3]:
def binarizeY(data):
    binarized_data = np.zeros((data.size,10))
    for j in range(0,data.size):
        feature = data[j:j+1]
        i = feature.astype(np.int64) 
        binarized_data[j,i]=1
    return binarized_data
train_labels_b = binarizeY(train_labels)
test_labels_b = binarizeY(test_labels)
numClasses = train_labels_b[1].size
print ('Classes = %d' %(numClasses))

Classes = 10


Lets start with a KNN model to establish a baseline accuracy.

Exercise: You've seen a number of different classification algorithms (e.g. naive bayes, decision trees, random forests, logistic regression) at this point.  How does KNN scalability and accuracy with respect to the size of the training dataset compare to those other algorithms?  

In [4]:
neighbors = 1
knn = KNeighborsClassifier(neighbors)
# we'll be waiting quite a while if we use 60K examples, so let's cut it down.  You may want to run the full 60K on your own later to see what the accuracy is.
mini_train_data, mini_train_labels = X[:numExamples], Y[:numExamples] 
start_time = time.time()
knn.fit(mini_train_data, mini_train_labels)
print ('Train time = %.2f' %(time.time() - start_time))
start_time = time.time()
accuracy = knn.score(test_data, test_labels)
print ('Accuracy = %.4f' %(accuracy))
print ('Prediction time = %.2f' %(time.time() - start_time))

Train time = 0.04
Accuracy = 0.9130
Prediction time = 4.82


Alright, now that we have a simple baseline, let's start working in Theano.  Before we jump to multi-layer neural networks though, let's train a logistic regression model to make certain we're using Theano correctly. 

Recall from Josh's regression lecture the four key components: (1) parameters, (2) model, (3) cost function, and (4) objective. 

In [5]:
## (1) Parameters 
# Initialize the weights to small, but non-zero, values.
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

Two notes relevant at this point:

First, logistic regression can be thought of as a neural network with no hidden layers. The output values are just the dot product of the inputs and the edge weights.

Second, we have 10 classes. We can either train separate one vs all classifiers using sigmoid activation, which would be a hassle, or we can use the softmax activation, which is essentially a multi-class version of sigmoid. We'll use Theano's built-in implementation of softmax.

In [6]:
## (2) Model
# Theano objects accessed with standard Python variables
X = T.matrix()
Y = T.matrix()

def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

We'll use cross-entropy as a cost function.  Cross entropy only considers the error between the true class and the prediction, and not the errors for the false classes.  This tends to cause the network to converge faster.  We'll use Theano's built-in cross entropy function.

In [7]:
## (3) Cost function
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

The objective is minimize the cost, and to do that we'll use batch gradient descent.

Exercise: What are the differences between batch, stochastic, and mini-batch gradient descent?  What are the implications of each for working on large datasets?

We'll use Theano's built-in gradient function.  Exercise: Do you recall from Josh's lecture what the gradient is for beta in logistic regression?

In [8]:
## (4) Objective (and solver)

alpha = 0.01
gradient = T.grad(cost=cost, wrt=w) 
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) # computes cost, then runs update
y_pred = T.argmax(y_hat, axis=1) # select largest probability as prediction
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

def gradientDescent(epochs):
    trainTime = 0.0
    predictTime = 0.0
    for i in range(epochs):
        start_time = time.time()
        cost = train(train_data[0:len(train_data)], train_labels_b[0:len(train_data)])
        trainTime =  trainTime + (time.time() - start_time)
        print ('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))
    print ('train time = %.2f' %(trainTime))

gradientDescent(50)

start_time = time.time()
predict(test_data)   
print ('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.1085
2) accuracy = 0.1370
3) accuracy = 0.1685
4) accuracy = 0.1955
5) accuracy = 0.2235
6) accuracy = 0.2565
7) accuracy = 0.2900
8) accuracy = 0.3265
9) accuracy = 0.3650
10) accuracy = 0.3980
11) accuracy = 0.4320
12) accuracy = 0.4620
13) accuracy = 0.4875
14) accuracy = 0.5130
15) accuracy = 0.5395
16) accuracy = 0.5610
17) accuracy = 0.5785
18) accuracy = 0.5910
19) accuracy = 0.6045
20) accuracy = 0.6090
21) accuracy = 0.6210
22) accuracy = 0.6265
23) accuracy = 0.6365
24) accuracy = 0.6445
25) accuracy = 0.6505
26) accuracy = 0.6555
27) accuracy = 0.6580
28) accuracy = 0.6625
29) accuracy = 0.6690
30) accuracy = 0.6730
31) accuracy = 0.6765
32) accuracy = 0.6810
33) accuracy = 0.6890
34) accuracy = 0.6915
35) accuracy = 0.6955
36) accuracy = 0.6970
37) accuracy = 0.7000
38) accuracy = 0.7030
39) accuracy = 0.7060
40) accuracy = 0.7075
41) accuracy = 0.7110
42) accuracy = 0.7115
43) accuracy = 0.7130
44) accuracy = 0.7145
45) accuracy = 0.7175
46) accuracy = 0.72

Exercise:  What do you expect to happen if we convert batch gradient descent to stochastic gradient descent?  Why?

Let's try it...

In [9]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 1 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print ('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data)))    ) 
    print ('train time = %.2f' %(trainTime))
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print ('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.8445
2) accuracy = 0.8640
3) accuracy = 0.8710
4) accuracy = 0.8720
5) accuracy = 0.8705
6) accuracy = 0.8695
7) accuracy = 0.8695
8) accuracy = 0.8685
9) accuracy = 0.8705
10) accuracy = 0.8715
11) accuracy = 0.8720
12) accuracy = 0.8710
13) accuracy = 0.8710
14) accuracy = 0.8720
15) accuracy = 0.8705
16) accuracy = 0.8700
17) accuracy = 0.8705
18) accuracy = 0.8725
19) accuracy = 0.8730
20) accuracy = 0.8730
21) accuracy = 0.8730
22) accuracy = 0.8725
23) accuracy = 0.8725
24) accuracy = 0.8735
25) accuracy = 0.8740
26) accuracy = 0.8735
27) accuracy = 0.8740
28) accuracy = 0.8735
29) accuracy = 0.8735
30) accuracy = 0.8730
31) accuracy = 0.8720
32) accuracy = 0.8720
33) accuracy = 0.8720
34) accuracy = 0.8720
35) accuracy = 0.8720
36) accuracy = 0.8720
37) accuracy = 0.8725
38) accuracy = 0.8730
39) accuracy = 0.8735
40) accuracy = 0.8735
41) accuracy = 0.8735
42) accuracy = 0.8740
43) accuracy = 0.8740
44) accuracy = 0.8735
45) accuracy = 0.8730
46) accuracy = 0.87

Exercise: What do you expect to happen if you switch the batch size to be great than 1 (i.e. mini-batch)?  Why?

Try it for a few values...

In [10]:
## (1) Parameters
w = theano.shared(np.asarray((np.random.randn(*(numFeatures, numClasses))*.01)))

## (2) Model
X = T.matrix()
Y = T.matrix()
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))
y_hat = model(X, w)

## (3) Cost
cost = T.mean(T.nnet.categorical_crossentropy(y_hat, Y))

## (4) Objective
alpha = 0.01
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * alpha]] 
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True) 
y_pred = T.argmax(y_hat, axis=1) 
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

miniBatchSize = 10 
def gradientDescentStochastic(epochs):
    trainTime = 0.0
    predictTime = 0.0
    start_time = time.time()
    for i in range(epochs):       
        for start, end in zip(range(0, len(train_data), miniBatchSize), range(miniBatchSize, len(train_data), miniBatchSize)):
            cost = train(train_data[start:end], train_labels_b[start:end])
        trainTime =  trainTime + (time.time() - start_time)
        print ('%d) accuracy = %.4f' %(i+1, np.mean(np.argmax(test_labels_b, axis=1) == predict(test_data))))     
    print ('train time = %.2f' %(trainTime))
    
gradientDescentStochastic(50)

start_time = time.time()
predict(test_data)   
print ('predict time = %.2f' %(time.time() - start_time))

1) accuracy = 0.8025
2) accuracy = 0.8310
3) accuracy = 0.8405
4) accuracy = 0.8525
5) accuracy = 0.8605
6) accuracy = 0.8635
7) accuracy = 0.8665
8) accuracy = 0.8665
9) accuracy = 0.8695
10) accuracy = 0.8720
11) accuracy = 0.8730
12) accuracy = 0.8740
13) accuracy = 0.8740
14) accuracy = 0.8775
15) accuracy = 0.8785
16) accuracy = 0.8780
17) accuracy = 0.8800
18) accuracy = 0.8800
19) accuracy = 0.8800
20) accuracy = 0.8810
21) accuracy = 0.8800
22) accuracy = 0.8815
23) accuracy = 0.8815
24) accuracy = 0.8825
25) accuracy = 0.8830
26) accuracy = 0.8835
27) accuracy = 0.8835
28) accuracy = 0.8835
29) accuracy = 0.8835
30) accuracy = 0.8845
31) accuracy = 0.8840
32) accuracy = 0.8840
33) accuracy = 0.8840
34) accuracy = 0.8850
35) accuracy = 0.8850
36) accuracy = 0.8850
37) accuracy = 0.8850
38) accuracy = 0.8860
39) accuracy = 0.8860
40) accuracy = 0.8865
41) accuracy = 0.8865
42) accuracy = 0.8865
43) accuracy = 0.8870
44) accuracy = 0.8870
45) accuracy = 0.8870
46) accuracy = 0.88